In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point

/Users/yanis/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df6 = pd.read_csv('velib-disponibilite-en-temps-reel-6h.csv',sep=";")
df7 = pd.read_csv('velib-disponibilite-en-temps-reel-7h.csv',sep=";")
df9 = pd.read_csv('velib-disponibilite-en-temps-reel-9h.csv',sep=";")
df10= pd.read_csv('velib-disponibilite-en-temps-reel-10h.csv',sep=";")
df11 = pd.read_csv('velib-disponibilite-en-temps-reel-11h.csv',sep=";")
df13 = pd.read_csv('velib-disponibilite-en-temps-reel-13h.csv',sep=";")
df14 = pd.read_csv('velib-disponibilite-en-temps-reel-14h.csv',sep=";")
df15 = pd.read_csv('velib-disponibilite-en-temps-reel-15h.csv',sep=";")
df16 = pd.read_csv('velib-disponibilite-en-temps-reel-16h.csv',sep=";")
df17 = pd.read_csv('velib-disponibilite-en-temps-reel-17h.csv',sep=";")
df18 = pd.read_csv('velib-disponibilite-en-temps-reel-18h.csv',sep=";")
df19 = pd.read_csv('velib-disponibilite-en-temps-reel-19h.csv',sep=";")
df00 = pd.read_csv('velib-disponibilite-en-temps-reel-23h59.csv',sep=";")
data = [df6,df7,df9,df10,df11,df13,df14,df15,df16,df17,df19,df00]

In [3]:
df = pd.concat(data, ignore_index=True, sort=False)
df

,Identifiant station,Nom station,Station en fonctionnement,Capacité de la station,Nombre bornettes libres,Nombre total vélos disponibles,Vélos mécaniques disponibles,Vélos électriques disponibles,Borne de paiement disponible,Retour vélib possible,Actualisation de la donnée,Coordonnées géographiques,Nom communes équipées,Code INSEE communes équipées,station_opening_hours
0,16107,Benjamin Godard - Victor Hugo,OUI,0,29,2,0,2,OUI,OUI,2025-03-13T06:00:24+00:00,"48.865983, 2.275725",Paris,75056,NaN
1,44015,Rouget de L'isle - Watteau,OUI,0,5,15,1,14,OUI,OUI,2025-03-13T06:00:54+00:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,94081,NaN
2,40001,Hôpital Mondor,OUI,0,10,18,10,8,OUI,OUI,2025-03-13T05:59:46+00:00,"48.798922410229, 2.4537451531298",Créteil,94028,NaN
3,9020,Toudouze - Clauzel,OUI,0,19,2,2,0,OUI,OUI,2025-03-13T05:53:08+00:00,"48.87929591733507, 2.3373600840568547",Paris,75056,NaN
4,14014,Jourdan - Stade Charléty,OUI,0,49,11,4,7,OUI,OUI,2025-03-13T05:58:13+00:00,"48.819428333369, 2.3433353751898",Paris,75056,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17647,40002,Bleuets - Bordières,OUI,0,14,7,4,3,OUI,OUI,2025-03-14T00:00:20+00:00,"48.802117531472, 2.4543687905029",Créteil,94028,NaN
17648,21302,Aristide Briand - Place de la Résistance,OUI,0,0,44,36,8,OUI,OUI,2025-03-14T00:01:56+00:00,"48.82124248401006, 2.2511002421379094",Issy-les-Moulineaux,92040,NaN
17649,15056,Place Balard,OUI,0,21,1,1,0,OUI,OUI,2025-03-13T23:56:44+00:00,"48.836395736424, 2.2784192115068",Paris,75056,NaN
17650,8004,Malesherbes - Place de la Madeleine,OUI,0,61,16,8,8,OUI,OUI,2025-03-14T00:00:07+00:00,"48.870406028483, 2.323243509808",Paris,75056,NaN


In [4]:
df.shape

(17652, 15)

In [5]:
df.dtypes

Identifiant station                object
Nom station                        object
Station en fonctionnement          object
Capacité de la station              int64
Nombre bornettes libres             int64
Nombre total vélos disponibles      int64
Vélos mécaniques disponibles        int64
Vélos électriques disponibles       int64
Borne de paiement disponible       object
Retour vélib possible              object
Actualisation de la donnée         object
Coordonnées géographiques          object
Nom communes équipées              object
Code INSEE communes équipées        int64
station_opening_hours             float64
dtype: object

In [6]:
df = df.drop(columns=['station_opening_hours'])


In [7]:
df["Capacité de la station"] = df["Nombre bornettes libres"]+df["Nombre total vélos disponibles"]
df

,Identifiant station,Nom station,Station en fonctionnement,Capacité de la station,Nombre bornettes libres,Nombre total vélos disponibles,Vélos mécaniques disponibles,Vélos électriques disponibles,Borne de paiement disponible,Retour vélib possible,Actualisation de la donnée,Coordonnées géographiques,Nom communes équipées,Code INSEE communes équipées
0,16107,Benjamin Godard - Victor Hugo,OUI,31,29,2,0,2,OUI,OUI,2025-03-13T06:00:24+00:00,"48.865983, 2.275725",Paris,75056
1,44015,Rouget de L'isle - Watteau,OUI,20,5,15,1,14,OUI,OUI,2025-03-13T06:00:54+00:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,94081
2,40001,Hôpital Mondor,OUI,28,10,18,10,8,OUI,OUI,2025-03-13T05:59:46+00:00,"48.798922410229, 2.4537451531298",Créteil,94028
3,9020,Toudouze - Clauzel,OUI,21,19,2,2,0,OUI,OUI,2025-03-13T05:53:08+00:00,"48.87929591733507, 2.3373600840568547",Paris,75056
4,14014,Jourdan - Stade Charléty,OUI,60,49,11,4,7,OUI,OUI,2025-03-13T05:58:13+00:00,"48.819428333369, 2.3433353751898",Paris,75056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17647,40002,Bleuets - Bordières,OUI,21,14,7,4,3,OUI,OUI,2025-03-14T00:00:20+00:00,"48.802117531472, 2.4543687905029",Créteil,94028
17648,21302,Aristide Briand - Place de la Résistance,OUI,44,0,44,36,8,OUI,OUI,2025-03-14T00:01:56+00:00,"48.82124248401006, 2.2511002421379094",Issy-les-Moulineaux,92040
17649,15056,Place Balard,OUI,22,21,1,1,0,OUI,OUI,2025-03-13T23:56:44+00:00,"48.836395736424, 2.2784192115068",Paris,75056
17650,8004,Malesherbes - Place de la Madeleine,OUI,77,61,16,8,8,OUI,OUI,2025-03-14T00:00:07+00:00,"48.870406028483, 2.323243509808",Paris,75056


In [8]:
df["Densite de la Station"] = (df["Nombre total vélos disponibles"]) / df["Capacité de la station"]
df

,Identifiant station,Nom station,Station en fonctionnement,Capacité de la station,Nombre bornettes libres,Nombre total vélos disponibles,Vélos mécaniques disponibles,Vélos électriques disponibles,Borne de paiement disponible,Retour vélib possible,Actualisation de la donnée,Coordonnées géographiques,Nom communes équipées,Code INSEE communes équipées,Densite de la Station
0,16107,Benjamin Godard - Victor Hugo,OUI,31,29,2,0,2,OUI,OUI,2025-03-13T06:00:24+00:00,"48.865983, 2.275725",Paris,75056,0.064516
1,44015,Rouget de L'isle - Watteau,OUI,20,5,15,1,14,OUI,OUI,2025-03-13T06:00:54+00:00,"48.778192750803, 2.3963020229163",Vitry-sur-Seine,94081,0.750000
2,40001,Hôpital Mondor,OUI,28,10,18,10,8,OUI,OUI,2025-03-13T05:59:46+00:00,"48.798922410229, 2.4537451531298",Créteil,94028,0.642857
3,9020,Toudouze - Clauzel,OUI,21,19,2,2,0,OUI,OUI,2025-03-13T05:53:08+00:00,"48.87929591733507, 2.3373600840568547",Paris,75056,0.095238
4,14014,Jourdan - Stade Charléty,OUI,60,49,11,4,7,OUI,OUI,2025-03-13T05:58:13+00:00,"48.819428333369, 2.3433353751898",Paris,75056,0.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17647,40002,Bleuets - Bordières,OUI,21,14,7,4,3,OUI,OUI,2025-03-14T00:00:20+00:00,"48.802117531472, 2.4543687905029",Créteil,94028,0.333333
17648,21302,Aristide Briand - Place de la Résistance,OUI,44,0,44,36,8,OUI,OUI,2025-03-14T00:01:56+00:00,"48.82124248401006, 2.2511002421379094",Issy-les-Moulineaux,92040,1.000000
17649,15056,Place Balard,OUI,22,21,1,1,0,OUI,OUI,2025-03-13T23:56:44+00:00,"48.836395736424, 2.2784192115068",Paris,75056,0.045455
17650,8004,Malesherbes - Place de la Madeleine,OUI,77,61,16,8,8,OUI,OUI,2025-03-14T00:00:07+00:00,"48.870406028483, 2.323243509808",Paris,75056,0.207792


In [9]:
df = df[df["Station en fonctionnement"] != "NON"]
df = df[df["Capacité de la station"] != 0]

In [10]:
df.to_csv('Velib-1303.csv', index=False)

In [11]:
df.count()

Identifiant station               17348
Nom station                       17348
Station en fonctionnement         17348
Capacité de la station            17348
Nombre bornettes libres           17348
Nombre total vélos disponibles    17348
Vélos mécaniques disponibles      17348
Vélos électriques disponibles     17348
Borne de paiement disponible      17348
Retour vélib possible             17348
Actualisation de la donnée        17348
Coordonnées géographiques         17348
Nom communes équipées             17348
Code INSEE communes équipées      17348
Densite de la Station             17348
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17348 entries, 0 to 17650
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Identifiant station             17348 non-null  object 
 1   Nom station                     17348 non-null  object 
 2   Station en fonctionnement       17348 non-null  object 
 3   Capacité de la station          17348 non-null  int64  
 4   Nombre bornettes libres         17348 non-null  int64  
 5   Nombre total vélos disponibles  17348 non-null  int64  
 6   Vélos mécaniques disponibles    17348 non-null  int64  
 7   Vélos électriques disponibles   17348 non-null  int64  
 8   Borne de paiement disponible    17348 non-null  object 
 9   Retour vélib possible           17348 non-null  object 
 10  Actualisation de la donnée      17348 non-null  object 
 11  Coordonnées géographiques       17348 non-null  object 
 12  Nom communes équipées           17348

In [13]:
df.describe()

,Capacité de la station,Nombre bornettes libres,Nombre total vélos disponibles,Vélos mécaniques disponibles,Vélos électriques disponibles,Code INSEE communes équipées,Densite de la Station
count,17348.000000,17348.000000,17348.000000,17348.000000,17348.000000,17348.000000,17348.000000
mean,30.514065,18.540985,11.973080,7.592057,4.381024,80977.835831,0.387655
std,11.546756,12.198867,11.739566,9.609142,4.758014,8406.504264,0.315826
min,7.000000,0.000000,0.000000,0.000000,0.000000,75056.000000,0.000000
25%,22.000000,9.000000,3.000000,1.000000,1.000000,75056.000000,0.105263
50%,28.000000,18.000000,8.000000,4.000000,3.000000,75056.000000,0.300000
75%,36.000000,26.000000,18.000000,12.000000,6.000000,92040.000000,0.656250
max,79.000000,65.000000,75.000000,67.000000,46.000000,95018.000000,1.000000


In [14]:
df.dropna(inplace=True)

In [15]:
print(df.isna().sum())  # Or df.isnull().sum() : it count the number of missing values per column

Identifiant station               0
Nom station                       0
Station en fonctionnement         0
Capacité de la station            0
Nombre bornettes libres           0
Nombre total vélos disponibles    0
Vélos mécaniques disponibles      0
Vélos électriques disponibles     0
Borne de paiement disponible      0
Retour vélib possible             0
Actualisation de la donnée        0
Coordonnées géographiques         0
Nom communes équipées             0
Code INSEE communes équipées      0
Densite de la Station             0
dtype: int64


I Think that we finished the cleaning of our dataset. There are no NaN or incoherent values, the columns names are understandable and evely columns have the same amount of datas with no missing values